# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this sql statement to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
```

#### Load Libraries

In [2]:
# Load Libraries

import os
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
#import flatiron_stats as fs

%matplotlib inline
plt.style.use("ggplot")

#### Connect to Dbase

In [14]:
# Connecting to database through sqlite3
conn = sqlite3.connect('./lego.db')
c = conn.cursor()

# List tables found in the database
response = c.execute("SELECT name FROM sqlite_master WHERE type = 'table' ORDER BY name;").fetchall()
response

[('product_details',),
 ('product_info',),
 ('product_pricing',),
 ('product_reviews',)]

#### Preview Data in database

In [6]:
product_info_df = pd.read_sql_query('SELECT * FROM product_info;', conn)
print(product_info_df.shape)
product_info_df.head()

(744, 4)


,prod_id,ages,piece_count,set_name
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate
2,7280,5-12,2,Straight & Crossroad Plates
3,7281,5-12,2,T-Junction & Curved Road Plates
4,7499,5-12,24,Flexible and Straight Tracks


In [7]:
product_pricing_df = pd.read_sql_query('SELECT * FROM product_pricing;', conn)
print(product_pricing_df.shape)
product_pricing_df.head()

(10870, 3)


,prod_id,country,list_price
0,75823,US,29.99
1,75822,US,19.99
2,75821,US,12.99
3,21030,US,99.99
4,21035,US,79.99


In [8]:
product_details_df = pd.read_sql_query('SELECT * FROM product_details;', conn)
print(product_details_df.shape)
product_details_df.head()

(744, 4)


,prod_id,prod_desc,prod_long_desc,theme_name
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®
2,7280,blah,blahblah,City
3,7281,blah,blahblah,City
4,7499,blah,blahblah,City


In [9]:
product_reviews_df = pd.read_sql_query('SELECT * FROM product_reviews;', conn)
print(product_reviews_df.shape)
product_reviews_df.head()

(744, 6)


,prod_id,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3
2,7280,85.0,4.1,Very Easy,3.5,2.3
3,7281,40.0,4.1,Very Easy,3.9,2.8
4,7499,89.0,2.9,Easy,2.5,2.2


#### Preview Tables

In [15]:
#Preview the tables:
for item in response:
    table = item[0]
    length = c.execute("""SELECT count(*) from {};""".format(table)).fetchall()
    results = c.execute("""SELECT * from {} limit 5;""".format(table)).fetchall()
    df = pd.DataFrame(results)
    df.columns = [x[0] for x in c.description]
    print(table, length, '\n', df, '\n\n')

product_details [(744,)] 
    prod_id prod_desc prod_long_desc theme_name
0      630      blah       blahblah    Classic
1     2304      blah       blahblah     DUPLO®
2     7280      blah       blahblah       City
3     7281      blah       blahblah       City
4     7499      blah       blahblah       City 


product_info [(744,)] 
    prod_id  ages  piece_count                         set_name
0      630    4+            1                  Brick Separator
1     2304  1½-5            1     LEGO® DUPLO® Green Baseplate
2     7280  5-12            2      Straight & Crossroad Plates
3     7281  5-12            2  T-Junction & Curved Road Plates
4     7499  5-12           24     Flexible and Straight Tracks 


product_pricing [(10870,)] 
    prod_id country list_price
0    75823      US      29.99
1    75822      US      19.99
2    75821      US      12.99
3    21030      US      99.99
4    21035      US      79.99 


product_reviews [(744,)] 
    prod_id  num_reviews  play_star_rating re

#### Join ALL tables 

In [17]:
query = """SELECT * FROM product_info
                  JOIN product_details
                  USING(prod_id)
                  JOIN product_pricing
                  USING(prod_id)
                  JOIN product_reviews
                  USING(prod_id);"""
result = c.execute(query).fetchall()
df = pd.DataFrame(result)
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

10870


,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,blah,blahblah,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,blah,blahblah,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,blah,blahblah,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,blah,blahblah,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,blah,blahblah,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1


#### info()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


#### describe()

In [19]:
df.describe()

,prod_id,piece_count,num_reviews,play_star_rating,star_rating,val_star_rating
count,1.087000e+04,10870.000000,9449.000000,9321.000000,9449.000000,9301.000000
mean,6.181634e+04,503.936431,17.813737,4.355413,4.510319,4.214439
std,1.736390e+05,831.209318,38.166693,0.617272,0.516463,0.670906
min,6.300000e+02,1.000000,1.000000,1.000000,1.800000,1.000000
25%,2.112300e+04,97.000000,2.000000,4.000000,4.300000,4.000000
50%,4.207350e+04,223.000000,6.000000,4.500000,4.600000,4.300000
75%,7.124800e+04,556.000000,14.000000,4.800000,5.000000,4.700000
max,2.000431e+06,7541.000000,367.000000,5.000000,5.000000,5.000000


#### Export data to .csv file

In [20]:
df.to_csv('Lego_data_merged.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!